In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from data.pricerunner import pricerunner_parser
from dataset import TreeSet, DumpedTreeSetWrapper
from feedback import GPTFeedback

OPENAI_API_KEY = None
GENERATE_DESCRIPTIONS = False

treeset = TreeSet([
    pricerunner_parser.getCategoryBrandProductTree(
        pricerunner_parser.cleanRows(
            pricerunner_parser.getRowsFromCSV('data/pricerunner/pricerunner_aggregate.csv')))], 
    OPENAI_API_KEY if GENERATE_DESCRIPTIONS else None)
dtsw = DumpedTreeSetWrapper(treeset)
gpt_feedback = GPTFeedback(OPENAI_API_KEY)

Parsed 35311 rows from data/pricerunner/pricerunner_aggregate.csv


In [27]:
sample = treeset[0]
print(sample)

In [29]:
sample_feedback = gpt_feedback.getCategories(sample[0], sample[1])
print(sample_feedback)

[{'product_name': 'New World NWDW60MK2 Integrated', 'category': 'Dishwashers/New'}, {'product_name': 'Miele G 6895 SCVi XXL K2O Stainless Steel', 'category': 'Dishwashers/Miele'}, {'product_name': 'YI M1 + 12-40mm + 42.5mm', 'category': 'Digital Cameras/YI'}, {'product_name': 'Hoover HCN6202XK Stainless Steel', 'category': 'Fridge Freezers/Hoover'}, {'product_name': 'Saba RF442BRPX6 Stainless Steel', 'category': 'Fridge Freezers/Saba'}, {'product_name': 'Essentials C50BS16', 'category': 'Fridge Freezers/Essentials'}, {'product_name': 'Land Rover Explore', 'category': 'Mobile Phones/Land'}, {'product_name': 'Lenovo Intel Xeon E5606 2.13GHz Socket 1366 1066MHz bus Upgrade Tray', 'category': 'CPUs/Lenovo'}, {'product_name': 'Bosch WUQ284F0', 'category': 'Washing Machines/Bosch'}, {'product_name': 'Swan SR11040CN Cream', 'category': 'Freezers/Swan'}]


In [43]:
from models import SmallT5

import torch
import trainer

model = SmallT5()
dataloader = torch.utils.data.DataLoader(dtsw, batch_size=2, shuffle=True, num_workers=True)
base_trainer = trainer.BaseTrainer(model, torch.optim.AdamW(model.parameters(), lr=3e-4), dataloader)

device = torch.device('cuda:0' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))

In [ ]:
#base_trainer.train(20)
model.load_state_dict(torch.load('smallt5_basic.pt').state_dict())

In [23]:
model.tokenizer.decode(out_tokens[0])

'<pad> [{ "product_name": "New World NWDW60MK'

In [ ]:
import copy

from trainer import PPOTrainer, ClipObjectiveLoss

ref_model = copy.deepcopy(model) 
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
objective = ClipObjectiveLoss()

ppo_trainer = PPOTrainer(model, ref_model, optimizer, dataloader, objective)

In [ ]:
ppo_trainer.train(max_length=16)

In [113]:
token = torch.Tensor([0, 0])
torch.all(token == torch.zeros(*token.shape))

tensor(True)